# Milestone 4
## Last Two Brain Cells

Group Members - 
 - Ananya Kulkarni
 - Souvik Saha

 - Purpose of this code is to segment the eeg epochs where music has been played to the subject.
 - The code then saves the eeg epochs in folders based on audio played
 - The code also calculates the average eeg of the no music played recordings and saves it

In [1]:
import mne, mne_bids
# mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib') # To plot in the notebook using matplotlib
import sys
sys.path.insert(0, '.')
import ccs_eeg_utils

import matplotlib.pyplot as plt

from mne import events_from_annotations
from mne_bids import BIDSPath, read_raw_bids
import numpy as np

import os

In [2]:
filtered_output_root = "/home/fsociety/Documents/Datasets/project-output"
epochs_output_root = "/home/fsociety/Documents/Datasets/project-epochs"

start_sub = 1
end_sub = 31

start_music = 300
end_music = 360

subjects = [f"{i:02d}" for i in range(start_sub, end_sub+1)]

no_music_run = ['run1', 'run6'] # no music
music_run = ['run2', 'run3', 'run4', 'run5'] # music
music_list = list(range(start_music, end_music+1)) 

runs = no_music_run + music_run # all runs

In [3]:
count = 0

no_music_epochs = []
for subject in subjects:
    for run in music_run:
        try:
            # Read the raw data (Music)
            bids_path = BIDSPath(subject=subject,task=run,
                                datatype='eeg', suffix='eeg',
                                root=filtered_output_root)
            raw = read_raw_bids(bids_path)
            ccs_eeg_utils.read_annotations_core(bids_path,raw)
            raw.load_data()
            raw.set_eeg_reference("average", verbose=False)
            raw.set_montage('standard_1020', match_case=False)
            
            events, event_id = events_from_annotations(raw)
            
            epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=12, baseline=None, event_repeated='merge', preload=True)
            
            for annot in raw.annotations.description:
                if annot.startswith("Piece of music played"):
                    try:
                        os.makedirs(f"{epochs_output_root}/{annot.split(':')[-1]}")
                    except FileExistsError as e:
                        pass
                    epochs[annot].save(f"{epochs_output_root}/{annot.split(':')[-1]}/{subject}_{run}_music-epo.fif", overwrite=True)
                    count += 1
                    
        except Exception as e:
            print(f"Error processing {subject} {run} {e}")
    
    for run in no_music_run:
        try:
            # Read the raw data (No music)
            bids_path = BIDSPath(subject=subject,task=run,
                                datatype='eeg', suffix='eeg',
                                root=filtered_output_root)
            raw = read_raw_bids(bids_path)
            ccs_eeg_utils.read_annotations_core(bids_path,raw)
            raw.load_data()
            raw.set_eeg_reference("average", verbose=False)
            raw.set_montage('standard_1020', match_case=False)
                        
            no_music_epochs.append(raw.get_data())
        except Exception as e:
            print(f"Error processing {subject} {run} {e}")
    
print(f"Processed {count} epochs")
                

Extracting EDF parameters from /home/fsociety/Documents/Datasets/project-output/sub-01/eeg/sub-01_task-run2_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /home/fsociety/Documents/Datasets/project-output/sub-01/eeg/sub-01_task-run2_events.tsv.
Reading channel info from /home/fsociety/Documents/Datasets/project-output/sub-01/eeg/sub-01_task-run2_channels.tsv.
Reading electrode coords from /home/fsociety/Documents/Datasets/project-output/sub-01/eeg/sub-01_space-CapTrak_electrodes.tsv.
Reading 0 ... 114799  =      0.000 ...   573.995 secs...
Used Annotations descriptions: ['Fixation cross displayed on screen:786', 'Question 01 presented to user (The music made me feel pleasent):800', 'Question 02 presented to user (The music made me feel energetic):801', 'Question 03 presented to user (The music made me feel tense):802', 'Question 04 presented to user (The music made me feel angry):803', 'Question 05 presented to user (The

In [14]:
# Calculate the average of the epochs and save it
no_music_epochs = mne.EpochsArray(no_music_epochs, info=raw.info)
average_no_audio = np.average(no_music_epochs.average().get_data(), axis=0).reshape(60800, 1)


Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated


In [16]:
np.save(f"{epochs_output_root}/average_no_audio.npy", average_no_audio)
print(f"Saved average_no_audio.npy")

Saved average_no_audio.npy
